In [2]:
# import functions

from pyspark.sql import SparkSession, SQLContext, Window
import pyspark.sql.functions as F
from pyspark.sql.types import *
import glob
import pandas as pd
spark = SparkSession.builder.getOrCreate()
import collections
from collections import Counter
from pyspark.sql import functions as F 
import matplotlib.pyplot as plt

import random
from math import sqrt
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import numpy as np
# Import plotting function
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed
from keras.layers import LSTM
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window
import matplotlib.dates as mdates

from tensorflow.keras.layers import Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from sklearn.metrics import accuracy_score
import tensorflow.keras.losses as loss

from sklearn.model_selection import train_test_split


sqlContext = SQLContext(spark)

c:\Users\ldronden\AppData\Local\pypoetry\Cache\virtualenvs\publieke-data-predictive-maintenance-6PaKd1Az-py3.9\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
data_path = '../data/'

PdM_telemetry = spark.read.csv(data_path + 'PdM_telemetry.csv', header=True)
PdM_machines = spark.read.csv(data_path + 'PdM_machines.csv', header=True)
PdM_failures = spark.read.csv(data_path + 'PdM_failures.csv', header=True)

In [4]:
df = PdM_telemetry.join(PdM_machines, ['machineID'], how='left')

df = df.join(PdM_failures, ['datetime', 'machineID'], how='left')

In [5]:
df = df.withColumn('datetime', F.to_timestamp('datetime'))
df = df.withColumn("machineID", F.col("machineID").cast(IntegerType())) 
df = df.withColumn("volt", F.col("volt").cast(DoubleType())) 
df = df.withColumn("rotate", F.col("rotate").cast(DoubleType())) 
df = df.withColumn("pressure", F.col("pressure").cast(DoubleType())) 
df = df.withColumn("vibration", F.col("vibration").cast(DoubleType())) 
df = df.withColumn("age", F.col("age").cast(IntegerType())) 

df = df.withColumn('failure', F.when(df.failure.isNull(), 0).otherwise(1))

In [6]:
w = Window.partitionBy('machineID').orderBy('datetime')

df = df.withColumn('cycle', F.rank().over(w))

## EDA

In [9]:
df = df.toPandas()

In [10]:
df.shape

(876142, 10)

In [11]:
test_split=round(len(df)*0.20)
test_split

175228

In [34]:
df = df.drop(columns='model')

In [35]:
df_for_training=df[:-175228]
df_for_testing=df[-175228:]

In [36]:
print(df_for_training.shape)
print(df_for_testing.shape)

(700914, 9)
(175228, 9)


In [37]:
df_for_training

,datetime,machineID,volt,rotate,pressure,vibration,age,failure,cycle
0,2015-01-01 06:00:00,28,184.307299,491.259532,106.203055,42.438195,1,0,1
1,2015-01-01 07:00:00,28,204.057451,448.828602,88.103328,42.225094,1,0,2
2,2015-01-01 08:00:00,28,163.617989,469.765913,105.329638,31.152873,1,0,3
3,2015-01-01 09:00:00,28,160.844715,448.495945,103.262838,39.525679,1,0,4
4,2015-01-01 10:00:00,28,194.185401,452.966573,110.873700,45.645514,1,0,5
...,...,...,...,...,...,...,...,...,...
700909,2016-01-01 03:00:00,98,180.410465,485.467071,117.467661,37.321110,20,0,8760
700910,2016-01-01 04:00:00,98,158.354201,389.828191,121.270784,38.201489,20,0,8761
700911,2016-01-01 05:00:00,98,193.754368,450.198921,127.851932,39.800055,20,0,8762
700912,2016-01-01 06:00:00,98,165.717790,501.520194,114.553412,37.696504,20,0,8763


In [38]:
df_for_training['datetime'] = pd.to_numeric(pd.to_datetime(df_for_training['datetime']))
df_for_testing['datetime'] = pd.to_numeric(pd.to_datetime(df_for_testing['datetime']))

C:\Users\ldronden\AppData\Local\Temp\ipykernel_28308\1040265715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_training['datetime'] = pd.to_numeric(pd.to_datetime(df_for_training['datetime']))
C:\Users\ldronden\AppData\Local\Temp\ipykernel_28308\1040265715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_testing['datetime'] = pd.to_numeric(pd.to_datetime(df_for_testing['datetime']))


In [39]:
scaler = MinMaxScaler(feature_range=(0,1))
df_for_training_scaled = scaler.fit_transform(df_for_training)
df_for_testing_scaled=scaler.transform(df_for_testing)

In [40]:
df_for_training_scaled

array([[0.00000000e+00, 2.72727273e-01, 5.51195141e-01, ...,
        5.00000000e-02, 0.00000000e+00, 0.00000000e+00],
       [1.14155251e-04, 2.72727273e-01, 6.76361584e-01, ...,
        5.00000000e-02, 0.00000000e+00, 1.14116170e-04],
       [2.28310502e-04, 2.72727273e-01, 4.20076796e-01, ...,
        5.00000000e-02, 0.00000000e+00, 2.28232341e-04],
       ...,
       [9.99885845e-01, 9.79797980e-01, 6.11065873e-01, ...,
        1.00000000e+00, 0.00000000e+00, 9.99771768e-01],
       [1.00000000e+00, 9.79797980e-01, 4.33384269e-01, ...,
        1.00000000e+00, 0.00000000e+00, 9.99885884e-01],
       [0.00000000e+00, 1.01010101e-01, 4.22746590e-01, ...,
        3.00000000e-01, 0.00000000e+00, 0.00000000e+00]])

In [41]:
print(df_for_training_scaled.shape)
print(df_for_testing_scaled.shape)


(700914, 9)
(175228, 9)


In [42]:
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,0])
    return np.array(dataX),np.array(dataY)      

In [43]:
trainX,trainY=createXY(df_for_training_scaled,30)

In [45]:
testX,testY=createXY(df_for_testing_scaled,30)


In [46]:
print("trainX Shape: ",trainX.shape)
print("trainY Shape: ",trainY.shape)

trainX Shape:  (700884, 30, 9)
trainY Shape:  (700884,)


In [47]:
print("testX Shape: ",testX.shape)
print("testY Shape: ",testY.shape)

testX Shape--  (175198, 30, 9)
testY Shape--  (175198,)


In [49]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [53]:
shape = trainX.shape[1]
feat_length = trainX.shape[2]

def build_model(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(50,return_sequences=True,input_shape=(shape, feat_length)))
    grid_model.add(LSTM(50))
    grid_model.add(Dropout(0.2))
    grid_model.add(Dense(1))

    grid_model.compile(loss = 'mse',optimizer = optimizer)
    return grid_model

grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(testX,testY))
parameters = {'batch_size' : [16,20],
              'epochs' : [8,10],
              'optimizer' : ['adam','Adadelta'] }

grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            cv = 2)

C:\Users\ldronden\AppData\Local\Temp\ipykernel_28308\851559994.py:14: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(testX,testY))


In [55]:
grid_search = grid_search.fit(trainX,trainY)

Epoch 1/8
21903/21903 [==============================] - 323s 15ms/step - loss: 9.7138e-04 - val_loss: 1.3748e-04
Epoch 2/8
21903/21903 [==============================] - 331s 15ms/step - loss: 6.0688e-04 - val_loss: 1.1263e-04
Epoch 3/8
21903/21903 [==============================] - 343s 16ms/step - loss: 5.9154e-04 - val_loss: 1.1973e-04
Epoch 4/8
21903/21903 [==============================] - 311s 14ms/step - loss: 5.8359e-04 - val_loss: 5.7473e-04
Epoch 5/8
21903/21903 [==============================] - 321s 15ms/step - loss: 5.7085e-04 - val_loss: 1.1716e-04
Epoch 6/8
21903/21903 [==============================] - 313s 14ms/step - loss: 5.7448e-04 - val_loss: 1.3926e-04
Epoch 7/8
21903/21903 [==============================] - 312s 14ms/step - loss: 5.6947e-04 - val_loss: 1.1817e-04
Epoch 8/8
21903/21903 [==============================] - 82s 4ms/step - loss: 1.4680e-04
Epoch 1/8
21903/21903 [==============================] - 375s 17ms/step - loss: 0.0011 - val_loss: 1.9416e-04
Epo

KeyboardInterrupt: 